In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.initializers import Constant
from tensorflow.keras import optimizers

In [0]:
from google.colab import drive
drive.mount('/gdrive')

#KERAS

In [0]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences

## prepare dataset

In [0]:
df = pd.read_csv("sentiment.csv")
df.head()

In [0]:
def get_word2id(df):
  vocab=set()
  for i in df.iteritems():
    words= i[1].split(' ')
    vocab.update(words)
  word2id= dict()
  id2word= dict()
  for i,word in enumerate(vocab):
    word2id[word]=i+1
    id2word[i+1] =word
  return word2id,id2word,vocab

In [0]:
def convert_data(df,word2id):
  new_dataset=[]
  for r in df.iteritems():
    new_dataset.append([word2id[w] for w in r[1].split(" ")])
  return new_dataset

In [0]:
word2id,id2word,vocab = get_word2id(df['sentence'])
new_dataset = convert_data(df['sentence'],word2id)

In [0]:
vocab_size = len(word2id)

In [0]:
new_dataset = pad_sequences(new_dataset, maxlen=128, dtype='int32', padding='post', truncating='post', value=0.0)


In [0]:
new_dataset = np.concatenate([new_dataset,df['polarity'].values.reshape(-1,1)],axis=1)

### split train , test

In [0]:
train = new_dataset[:int(len(new_dataset)*.8)]
test = new_dataset[int(len(new_dataset)*.8):]

### evaluation

In [0]:
def evaluate(y_true,y_pred):
  return confusion_matrix(y_true, y_pred, labels=[0,1])

## Sentiment Analysis with Fully-Connected Neural Network (keras lubrary)

In [0]:
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation=tf.nn.relu))
model.add(keras.layers.Dense(16,activation=tf.nn.relu))
model.add(keras.layers.Dense(2,activation=tf.nn.softmax))
model.summary()

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x = train[:,:-1],y=train[:,-1],batch_size=32,epochs=5,validation_split=0.2)